## 1-Read Data and Process

In [1]:
from glob import glob
import os
import numpy as np
import pandas
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
!pip install mne

In [3]:
import mne

In [4]:
glob('EEG_Data/*edf')

['EEG_Data\\h01.edf',
 'EEG_Data\\h02.edf',
 'EEG_Data\\h03.edf',
 'EEG_Data\\h04.edf',
 'EEG_Data\\h05.edf',
 'EEG_Data\\h06.edf',
 'EEG_Data\\h07.edf',
 'EEG_Data\\h08.edf',
 'EEG_Data\\h09.edf',
 'EEG_Data\\h10.edf',
 'EEG_Data\\h11.edf',
 'EEG_Data\\h12.edf',
 'EEG_Data\\h13.edf',
 'EEG_Data\\h14.edf',
 'EEG_Data\\s01.edf',
 'EEG_Data\\s02.edf',
 'EEG_Data\\s03.edf',
 'EEG_Data\\s04.edf',
 'EEG_Data\\s05.edf',
 'EEG_Data\\s06.edf',
 'EEG_Data\\s07.edf',
 'EEG_Data\\s08.edf',
 'EEG_Data\\s09.edf',
 'EEG_Data\\s10.edf',
 'EEG_Data\\s11.edf',
 'EEG_Data\\s12.edf',
 'EEG_Data\\s13.edf',
 'EEG_Data\\s14.edf']

In [5]:
all_file_path=glob('EEG_Data/*edf')
print(len(all_file_path))

28


In [6]:
all_file_path[0]

'EEG_Data\\h01.edf'

In [7]:
healthy_file_path =[i for i in all_file_path if 'h' in i.split('\\')[1] ]
patient_file_path =[i for i in all_file_path if 's' in i.split('\\')[1] ]
print (len(healthy_file_path),len(patient_file_path ))



14 14


In [8]:
def read_data(file_path):
    data=mne.io.read_raw_edf(file_path, preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5,h_freq=45)
    #data.filter(l_freq=0.1,h_freq=60)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    #epochs=mne.make_fixed_length_epochs(data,duration=3,overlap=1)
    array=epochs.get_data()
    return array


In [9]:
sample_data=read_data(healthy_file_path[0])

Extracting EDF parameters from D:\notebooks\EEG_Pro\EEG_Data\h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 sec)

Not setting metadata
Not setting metadata
231 matching events found
No baseline correction applied
0 projection

In [10]:
sample_data.shape            #no of epochs, channels, Length of signal

(231, 19, 1250)

In [11]:
%%capture
control_epochs_array=[read_data(i) for i in healthy_file_path]
patient_epochs_array=[read_data(i) for i in patient_file_path]

In [12]:
control_epochs_array[0].shape,control_epochs_array[1].shape

((231, 19, 1250), (227, 19, 1250))

In [13]:
control_epochs_labels=[len(i)*[0] for i in control_epochs_array]
patient_epochs_labels=[len(i)*[1] for i in patient_epochs_array]
len(control_epochs_labels) , len(patient_epochs_labels)

(14, 14)

In [14]:
epochs_array=control_epochs_array+patient_epochs_array
epochs_labels=control_epochs_labels+patient_epochs_labels
print(len(epochs_array), len(epochs_labels))

28 28


In [15]:
data_list=control_epochs_array+patient_epochs_array
label_list=control_epochs_labels+patient_epochs_labels


In [16]:
group_list=[[i]*len(j) for i,j in enumerate(data_list)]
len(group_list)

28

In [17]:
data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(group_list)
print(data_array.shape,label_array.shape, group_array.shape)

(7201, 19, 1250) (7201,) (7201,)


## Machine Learning Classification

In [18]:
from scipy import stats
def mean(x):
    return np.mean(x,axis=-1) 
def std(x):
    return np.std(x,axis=-1) 
def ptp(x): 
    return np.ptp(x,axis=-1) 
def var(x):
     return np.var(x,axis=1)
def minim(x):
    return np.min(x,axis=-1) 
def maxim(x): 
    return np.max(x,axis=-1) 
def argminim(x):
    return np.argmin(x,axis=-1) 
def argmaxim(x):
    return np.argmax(x,axis=-1) 
def rms(x): 
    return np.sqrt(np.mean(x**2,axis=-1)) 
def abs_diff_signal(x):
    return np.sum(np.abs(np.diff(x,axis=-1)),axis=-1)
def skewness(x):
    return stats.skew(x,axis=-1) 
def kurtosis(x):
    return stats.kurtosis(x,axis=-1) 
def concatenate_features(x): 
    return np.concatenate((mean(x),std(x),ptp(x),var(x),minim(x),maxim(x),argminim(x), 
           argmaxim(x),rms(x),abs_diff_signal(x),skewness(x),kurtosis(x)),axis=-1) 

In [19]:
#mean(d).shape 

In [20]:
#std(d).shape 

In [21]:
#abs_diff_signal(d).shape 

In [22]:
features=[]
for d in data_array:
    features.append(concatenate_features(d)) 

In [23]:
features_array=np.array(features)
features_array.shape 

(7201, 228)

In [24]:
from sklearn.linear_model import LogisticRegression 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV 

In [25]:
clf=LogisticRegression() 
gkf=GroupKFold(5) 
pipe=Pipeline([('scalers',StandardScaler()),('clf',clf)]) 
param_grid={'clf__C':[0.1,0.5,0.7,1,3,5,7]} 
#param_grid = {'n__neighbors': np.arange(1, 12, 2)}
gscv=GridSearchCV(pipe,param_grid,cv=gkf,n_jobs=12) 
gscv.fit(features_array, label_array,groups=group_array) 
#param_grid = {'n_neighbors','clf__C': np.arange(1, 12, 2)}

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=Pipeline(steps=[('scalers', StandardScaler()),
                                       ('clf', LogisticRegression())]),
             n_jobs=12, param_grid={'clf__C': [0.1, 0.5, 0.7, 1, 3, 5, 7]})

In [26]:
print("Best params:", gscv.best_params_)
print("Best score:", gscv.best_score_)

Best params: {'clf__C': 0.7}
Best score: 0.6631887188110056


## Deep learning Classification using deep 1D convolutional neural network

In [27]:
import tensorflow as tf 
# physical_devices = tf.config.list_physical_devices('GPU') 
# tf.config.experimental.set_memory_growth(physical_devices[0], True) 


In [28]:
epochs_array =np.vstack(epochs_array) 
epochs_labels=np.hstack(epochs_labels)
groups_array=np.hstack(group_list) 

In [29]:
epochs_array.shape,epochs_labels.shape,groups_array.shape 



((7201, 19, 1250), (7201,), (7201,))

In [30]:
epochs_array=np.moveaxis(epochs_array,1,2)
epochs_array.shape

(7201, 1250, 19)

In [31]:
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D 
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session 
def cnnmodel():
    clear_session() 
    model=Sequential()
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(1250,19)))#1 
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#2 
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3 
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#4
    model.add(Dropout(0.5)) 
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    model.add(LeakyReLU()) 
    model.add(AveragePooling1D(pool_size=2,strides=2))#6 
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    model.add(LeakyReLU()) 
    model. add(AveragePooling1D(pool_size=2, strides=2))#8 
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9 
    model.add(LeakyReLU()) 
    model.add(GlobalAveragePooling1D())#10
    model.add(Dense(1,activation='sigmoid'))#11 
    model.compile('adam',loss='binary_crossentropy',metrics=['accuracy']) 
    return model 

model=cnnmodel() 
model. summary() 



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1248, 5)           290       
_________________________________________________________________
batch_normalization (BatchNo (None, 1248, 5)           20        
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 1248, 5)           0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 624, 5)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 622, 5)            80        
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 622, 5)            0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 311, 5)            0

In [32]:
from sklearn.model_selection import GroupKFold, LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold() 



In [33]:
import numpy as np

img = np.random.random((28, 28))
image = np.array(img, dtype='float64') / 255.
image = np.expand_dims(image, axis=0)
image = image.reshape(image.shape + (1,))

In [40]:
accuracy=[] 
for train_index,val_index in gkf.split(epochs_array,epochs_labels,groups=groups_array): 
    train_features,train_labels=epochs_array[train_index],epochs_labels[train_index] 
    val_features,val_labels=epochs_array[val_index],epochs_labels[val_index] 
    scaler=StandardScaler() 
    train_features=scaler.fit_transform(train_features.reshape(-1,train_features.shape[-1])).reshape(train_features.shape)
    val_features=scaler.transform(val_features.reshape(-1,val_features.shape[-1])).reshape(val_features.shape)
                            #val_features.shape[-1])).reshape(val_features.shape)
    model=cnnmodel() 
    model.fit(train_features,train_labels,epochs=10,validation_data=(val_features,val_labels)) 
    accuracy.append(model.evaluate(val_features,val_labels)[1])
                                                                           

Epoch 1/10
180/180 [==============================] - 20s 105ms/step - loss: 0.6627 - accuracy: 0.5822 - val_loss: 0.6890 - val_accuracy: 0.3734
Epoch 2/10
180/180 [==============================] - 13s 71ms/step - loss: 0.5701 - accuracy: 0.7006 - val_loss: 0.4519 - val_accuracy: 0.8243
Epoch 3/10
180/180 [==============================] - 10s 53ms/step - loss: 0.4704 - accuracy: 0.8031 - val_loss: 0.4568 - val_accuracy: 0.7872
Epoch 4/10
180/180 [==============================] - 19s 108ms/step - loss: 0.4386 - accuracy: 0.8221 - val_loss: 0.3886 - val_accuracy: 0.8415
Epoch 5/10
180/180 [==============================] - 4s 24ms/step - loss: 0.4050 - accuracy: 0.8388 - val_loss: 0.3990 - val_accuracy: 0.8435
Epoch 6/10
180/180 [==============================] - 3s 16ms/step - loss: 0.3781 - accuracy: 0.8534 - val_loss: 0.3922 - val_accuracy: 0.8250
Epoch 7/10
180/180 [==============================] - 3s 16ms/step - loss: 0.3492 - accuracy: 0.8708 - val_loss: 0.3940 - val_accuracy: 

In [41]:
np.mean(accuracy)

0.7182102084159852

In [42]:
print(accuracy)  # additional

[0.8490048050880432, 0.6113743782043457, 0.614374041557312, 0.7814391255378723, 0.7348586916923523]
